In [1]:
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from itertools import chain
import random
from copy import copy
import numpy as np
from dataclasses import dataclass, field

In [2]:
! ls -lh

total 91728
-rw-r--r--  1 Alexander  staff    23K Apr 27 19:38 2sat.ipynb
-rw-r--r--  1 Alexander  staff   1.2M Apr 27 03:28 2sat1.txt
-rw-r--r--  1 Alexander  staff   2.6M Apr 27 03:28 2sat2.txt
-rw-r--r--  1 Alexander  staff   5.5M Apr 27 03:28 2sat3.txt
-rw-r--r--  1 Alexander  staff   8.4M Apr 27 03:28 2sat4.txt
-rw-r--r--  1 Alexander  staff    11M Apr 27 03:28 2sat5.txt
-rw-r--r--  1 Alexander  staff    14M Apr 27 03:28 2sat6.txt
-rw-r--r--  1 Alexander  staff   4.5K Apr 27 19:02 papa_2sat.py
-rw-r--r--  1 Alexander  staff   131B Apr 27 18:16 test1_false.txt
-rw-r--r--  1 Alexander  staff   264B Apr 27 18:16 test2_true.txt


In [3]:
with open('2sat1.txt') as file:
    lines = file.read().splitlines()

In [4]:
lines[:5]

['100000', '-16808 75250', '43659 8931', '-27545 -50879', '-37710 64441']

In [5]:
'Number of vars', len(set([abs(int(x)) for x in ' '.join(lines[1:]).split(' ')]))

('Number of vars', 86569)

# Papadimitrion

In [6]:
@dataclass
class Clause:
    first: int
    second: int

        
class Formula:
    def __init__(self, clauses):
        self.clauses = clauses
        
        self.vs = set()
        for c in self.clauses:
            self.vs.add(abs(c.first))
            self.vs.add(abs(c.second))
        self.n_vars = len(self.vs)
        print('Number of variables:', self.n_vars)
        self.D = dict()
        
        
    def init_random_sol(self):
        for v in self.vs:
            self.D[v] = bool(random.getrandbits(1))

            
    def switch_var(self, literal):
        self.D[abs(literal)] = not self.D[abs(literal)]
    
    
    def check_literal(self, literal):
        if literal > 0:
            return self.D[literal]
        else:
            return not self.D[-literal]
    
    
    def check_clause(self, clause):
        return self.check_literal(clause.first) \
            or self.check_literal(clause.second)
    
    
    def get_false_clauses(self):
        return [x for x in self.clauses if self.check_clause(x) == False]

In [7]:
# https://github.com/ChuntaoLu/Algorithms-Design-and-Analysis/blob/eedab75c5dd2fafd1123150209d174cf6f64914f/week12%20local%20search%20and%202SAT%20problem/papa_2sat.py#L43
def reduce_clause(all_clause_pairs):
    singular_var = set()
    clause_var_dict = {} 
    var_clause_dict = defaultdict(set)
    for x, y in all_clause_pairs:
        var_clause_dict[x].add((x, y))
        var_clause_dict[y].add((x, y))
        clause_var_dict[(x,y)] = [x, y]
    while True:
        for var in singular_var:
            for clause in var_clause_dict[var].copy():
                del clause_var_dict[clause]
                var_clause_dict[clause[0]] -= set([clause])
                var_clause_dict[clause[1]] -= set([clause])
        reduced_var = set(chain(*clause_var_dict.values()))
        singular_var = set([i for i in reduced_var if -i not in reduced_var])
        if singular_var == set():
            break
    return set(clause_var_dict.keys())


def Papadimitrion_2SAT(formula):
    for _ in tqdm(range(int(np.log2(formula.n_vars)))):
        formula.init_random_sol()
        for _ in tqdm(range(2 * formula.n_vars**2)):
            false_clauses = formula.get_false_clauses()
            if len(false_clauses) == 0: 
                return True
            chosen_clause = random.choice(false_clauses)
            chosen_var = abs(random.choice((chosen_clause.first, chosen_clause.second)))
            formula.switch_var(chosen_var)
    return False

In [8]:
with open('2sat1.txt') as file:
    lines = file.read().splitlines()
clauses = []
for line in lines[1:]:
    a, b = tuple(line.split(' '))
    clauses.append( Clause(first=int(a), second=int(b)) )

print('Number of clauses before:', len(clauses))
clauses = reduce_clause( set((x.first, x.second) for x in clauses) )
clauses = [Clause(x[0], x[1]) for x in clauses]
print('Number of clauses after:', len(clauses))
formula = Formula(clauses)

Number of clauses before: 100000
Number of clauses after: 6
Number of variables: 6


In [9]:
bool_res = Papadimitrion_2SAT(formula)
bool_res

True

In [10]:
for i in range(1,7):
    with open(f'2sat{i}.txt') as file:
        lines = file.read().splitlines()
    clauses = []
    for line in lines[1:]:
        a, b = tuple(line.split(' '))
        clauses.append( Clause(first=int(a), second=int(b)) )

    print('Number of clauses before:', len(clauses))
    clauses = reduce_clause( set((x.first, x.second) for x in clauses) )
    clauses = [Clause(x[0], x[1]) for x in clauses]
    print('Number of clauses after:', len(clauses))
    formula = Formula(clauses)
    bool_res = Papadimitrion_2SAT(formula)
    print(i, bool_res)

Number of clauses before: 100000
Number of clauses after: 6
Number of variables: 6




1 True
Number of clauses before: 200000
Number of clauses after: 57
Number of variables: 54




2 False
Number of clauses before: 400000
Number of clauses after: 295
Number of variables: 288




3 True
Number of clauses before: 600000
Number of clauses after: 11
Number of variables: 11




4 True
Number of clauses before: 800000
Number of clauses after: 101
Number of variables: 98




5 False
Number of clauses before: 1000000
Number of clauses after: 26
Number of variables: 25




6 False


small cases

In [11]:
# for file_name in ['test1_false.txt', 'test2_true.txt']:
#     with open(file_name) as file:
#         lines = file.read().splitlines()
#     n_vars = int(lines[0])
#     clauses = []
#     for line in lines[1:]:
#         a, b = tuple(line.split(' '))
#         clauses.append( Clause(first=int(a), second=int(b)) )

#     formula = Formula(n_vars=n_vars, clauses=clauses, D={})
#     bool_res = Paradimitrion_2SAT(formula, int(np.log2(formula.n_vars)))
#     print(file_name, bool_res)